# Ejercicio. 

Este ejercicio es individualizado. Cada uno parte del número de su DNI, supongamos por ejemplo 12340987. Dividimos dicho número en dos bloques, 1234 y 0987. Si alguno de ellos es menor que 1000, rotamos las cifras a la izquierda hasta obtener un número mayor o igual que 1000, en nuestro caso 9870 (si alguien tuviese como un bloque el 0000, que coja un número mayor que 1000 cualquiera a su elección.

Sean p y q los primeros primos mayores o iguales que los bloques anteriores. Concretamente, en el ejemplo p = 1237 y q = 9871. Sea n = pq y e el menor primo mayor o igual que 11 que es primo relativo con φ(n). Sea d = e^−1 mod φ(n).

1. Cifra el mensaje m = 0xCAFE (recordad que 0x indica que el número está escrito en hexadecimal).
2. Descifra el criptograma anterior.
3. Intenta factorizar n mediante el método P − 1 de Pollard. Para ello llega, como máximo a b = 8.
4. Intenta factorizar n a partir de φ(n).
5. Intenta factorizar n a partir de e y d

Primero, vamos a preparar los datos necesarios para el ejercicio

In [4]:
dni = 77432071
p = next_prime(7743)
q = next_prime(2071)

p, q

(7753, 2081)

In [5]:
n = p*q

In [6]:
e = 11
p = 11
primes = Primes()

while p.gcd(euler_phi(n)) != 1:
    p = primes.next(p)

e = p
e

11

In [7]:
d = inverse_mod(e, euler_phi(n))
d

5863331

¿Lo hemos calculado bien?

In [8]:
(e * d) % euler_phi(n)

1

## Apartado 1

In [9]:
m = 0xCAFE

In [10]:
def cifrar(m, e, n):
    return power_mod(m, e, n)

In [11]:
c = cifrar(m, e, n)
c

10576233

En hexa:

In [12]:
'0x' + c.hex()

'0xa16169'

## Apartado 2

In [13]:
def descifrar(c, e, n):
    d = inverse_mod(e, euler_phi(n))
    return power_mod(c, d, n)

In [14]:
m = descifrar(c, e, n)
'0x' + m.hex()

'0xcafe'

## Apartado 3

In [54]:
b_valores = list(range(2, 9))
b_valores

def p_1_Pollard(n, valores_para_b = [2, 3, 4, 5, 6, 7, 8]):
    # Buscamos la base 
    
    base = 2 
    for i in range(2, n):
        if n.gcd(i) == 1:
            base = i
            break
    
    print("Encontrada base " + str(base))
    
    encontrado = False
    
    for b in valores_para_b:
        powermod = power_mod(base, factorial(b), n)
        mcd = gcd(powermod - 1, n)
        print(str(base) + "^" + str(b) + "! mod n = " + str(powermod) + " => gcd(" + str(powermod) + " - 1, n) = " + str(mcd))
        
        if mcd != 1:
            encontrado = True
            # return []?

    if not encontrado:
        return False
    

        

In [55]:
p_1_Pollard(n)

Encontrada base 2
2^2! mod n = 4 => gcd(4 - 1, n) = 1
2^3! mod n = 64 => gcd(64 - 1, n) = 1
2^4! mod n = 643223 => gcd(643223 - 1, n) = 1
2^5! mod n = 4648376 => gcd(4648376 - 1, n) = 1
2^6! mod n = 9022979 => gcd(9022979 - 1, n) = 1
2^7! mod n = 13516495 => gcd(13516495 - 1, n) = 1
2^8! mod n = 10417810 => gcd(10417810 - 1, n) = 1


False

No hemos sido capaces de factorizar $n$ en 8 pasos esta vez

## Apartado 4 

Tomamos el polinomio $(x - p)(x - q) = x^2 - (p + q)x + n$. Puesto que $\varphi(n) = n + 1 - (p + q)$, se tiene que 

$$
(x - p)(x - q) = x^2 - (n + 1 - \varphi(n))x + n
$$

Claramente, las raíces del polinomio son $p$ y $q$, por lo que podemos hallar sus valores resolviendo la ecuación 

$$
\begin{cases}
p, q = b \pm \sqrt{b^2 - n} \\
p + q = 2b \\
p + q = n - \varphi(n) + 1 
\end{cases}
$$

Así que 

$$
b = \frac{n - \varphi(n) + 1 }{2}
$$

In [16]:
b = (n - euler_phi(n) + 1)/2
b

4917

In [17]:
p = b + sqrt(b^2 - n)
q = b - sqrt(b^2 - n)
p, q

(7753, 2081)

Por tanto, hemos hallado los valores de $p$ y $q$

## Apartado 5

Queremos factorizar $n$ a partir de $e$ y $d$ esta vez.

Para ello, consideramos los primeros primos hasta 19, y buscamos el primer $a^k \pmod n$ distinto de 1. Reducimos k a la mitad cada vez que pasemos a otra itercación.

In [20]:
def factorizar_n(e, d, n):
    k = e*d - 1
    
    debemos_parar = False
    
    while not debemos_parar:
        for a in [2, 3, 5, 7, 11, 13, 17, 19]:
            actual = power_mod(a, k, n)
            print(str(a) + "^" + str(k) + " mod n = " + str(actual))
            
            if actual != 1:
                debemos_parar = True
                solucion = actual
                break
        
        k = k // 2
        print("\n")
    
    return [n.gcd(solucion - 1), n.gcd(solucion + 1)]

In [21]:
factorizar_n(e, d, n)

2^64496640 mod n = 1
3^64496640 mod n = 1
5^64496640 mod n = 1
7^64496640 mod n = 1
11^64496640 mod n = 1
13^64496640 mod n = 1
17^64496640 mod n = 1
19^64496640 mod n = 1


2^32248320 mod n = 1
3^32248320 mod n = 1
5^32248320 mod n = 1
7^32248320 mod n = 1
11^32248320 mod n = 1
13^32248320 mod n = 1
17^32248320 mod n = 1
19^32248320 mod n = 1


2^16124160 mod n = 1
3^16124160 mod n = 1
5^16124160 mod n = 1
7^16124160 mod n = 1
11^16124160 mod n = 1
13^16124160 mod n = 1
17^16124160 mod n = 1
19^16124160 mod n = 1


2^8062080 mod n = 1
3^8062080 mod n = 1
5^8062080 mod n = 1
7^8062080 mod n = 1
11^8062080 mod n = 1
13^8062080 mod n = 1
17^8062080 mod n = 1
19^8062080 mod n = 1


2^4031040 mod n = 1
3^4031040 mod n = 1
5^4031040 mod n = 1
7^4031040 mod n = 1
11^4031040 mod n = 1
13^4031040 mod n = 1
17^4031040 mod n = 1
19^4031040 mod n = 1


2^2015520 mod n = 1
3^2015520 mod n = 1
5^2015520 mod n = 1
7^2015520 mod n = 1
11^2015520 mod n = 1
13^2015520 mod n = 1
17^2015520 mod n = 1
19^

[7753, 2081]

Así que hemos conseguido factorizar $n$ a partir de $e$ y $d$